<a href="https://colab.research.google.com/github/Storm00212/Data-science-and-ml-resource/blob/main/Understanding_RAG_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers faiss-cpu transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 39.9 MB/s eta 0:00:00


In [2]:
documents = [
    "Kenya’s capital city is Nairobi.",
    "A RAG pipeline retrieves documents before generating an answer.",
    "The speed of light is approximately 3×10^8 meters per second.",
    "Python is widely used for machine learning and AI projects."
]


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Create embeddings
embeddings = model.encode(documents)

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

print("Indexed documents:", index.ntotal)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Indexed documents: 4


In [4]:
def retrieve(query, k=2):
    query_vec = model.encode([query])
    distances, indices = index.search(np.array(query_vec), k)
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

gen_model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
generator = AutoModelForCausalLM.from_pretrained(gen_model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
def rag_answer(query):
    # Step 1: Retrieve
    context = retrieve(query, k=2)
    context_text = "\n".join(context)

    # Step 2: Construct prompt
    prompt = f"Context:\n{context_text}\n\nQuestion: {query}\nAnswer:"

    # Step 3: Generate answer
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = generator.generate(**inputs, max_length=150)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer


In [7]:
query = "What does a RAG pipeline do?"
print(rag_answer(query))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Context:
A RAG pipeline retrieves documents before generating an answer.
Python is widely used for machine learning and AI projects.

Question: What does a RAG pipeline do?
Answer:
A RAG pipeline retrieves documents before generating an answer.
Python is widely used for machine learning and AI projects.
Question: What does a RAG pipeline do?
Answer:
A RAG pipeline retrieves documents before generating an answer.
Python is widely used for machine learning and AI projects.
Question: What does a RAG pipeline do?
Answer:
A RAG pipeline retrieves documents before generating an answer.
Python is widely used for machine learning and AI projects.
Question: What does
